In [4]:
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage, AIMessage
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from typing import Annotated
from typing_extensions import TypedDict

api_key = "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"
llm = ChatGroq(groq_api_key=api_key, model_name="llama-3.3-70b-versatile")

class State(TypedDict):
    messages: Annotated[list, add_messages]
    name: str
    standard: str
    topic: str

graph_builder = StateGraph(State)

def icebreaker(state: State):
    name = state.get("name", "student")
    topic = state.get("topic", "learning")
    student_msg = state["messages"][-1].content

    system_prompt = (
        f"Scenario: A student named {name} is starting to learn about {topic}.\n"
        f"Student ({name}): {student_msg}\n"
        f"Initial Icebreaker Agent: Respond like a warm and engaging tutor starting a fun lesson. "
        f"Make the student feel welcome and excited to learn about {topic}. End with a friendly question."
    )
    response = llm.invoke([HumanMessage(content=system_prompt)])
    return {"messages": [AIMessage(content=response.content)]}

def definition_node(state: State):
    topic = state["messages"][-1].content
    prompt = f"You are a teacher. Give a simple and clear definition for: {topic}"
    response = llm.invoke([HumanMessage(content=prompt)])
    return {"messages": [AIMessage(content=response.content)]}

def quiz_node(state: State):
    topic = state["messages"][-1].content
    prompt = f"You are a teacher. Ask a short, fun quiz question related to: {topic}"
    response = llm.invoke([HumanMessage(content=prompt)])
    return {"messages": [AIMessage(content=response.content)]}

def example_node(state: State):
    topic = state["messages"][-1].content
    prompt = f"You are a teacher. Give simple real-life examples to help a student understand: {topic}"
    response = llm.invoke([HumanMessage(content=prompt)])
    return {"messages": [AIMessage(content=response.content)]}

def needs_assessment(state: State):
    prompt = (
        "Ask the student what part they are struggling with. Be friendly and offer to help with specific doubts."
    )
    response = llm.invoke([HumanMessage(content=prompt)])
    return {"messages": [AIMessage(content=response.content)]}

def router(state: State) -> dict:
    user_input = state["messages"][-1].content
    prompt = (
        "You are a smart router for an AI Tutoring System. "
        "Given the student's input, choose the most appropriate response node from the following options: "
        "'icebreaker', 'definition', 'quiz', 'examples', or 'needs_assessment'.\n"
        f"Student input: \"{user_input}\"\n"
        "Return only the one word corresponding to the next node."
    )
    response = llm.invoke([HumanMessage(content=prompt)])
    next_node = response.content.strip().lower()
    valid_options = ['icebreaker', 'definition', 'quiz', 'examples', 'needs_assessment']
    if next_node not in valid_options:
        next_node = "needs_assessment"
    return {"next": next_node}

graph_builder.add_node("router", router)
graph_builder.add_node("icebreaker", icebreaker)
graph_builder.add_node("definition", definition_node)
graph_builder.add_node("quiz", quiz_node)
graph_builder.add_node("examples", example_node)
graph_builder.add_node("needs_assessment", needs_assessment)

graph_builder.set_entry_point("router")
graph_builder.add_conditional_edges("router", lambda x: x["next"], {
    "icebreaker": "icebreaker",
    "definition": "definition",
    "quiz": "quiz",
    "examples": "examples",
    "needs_assessment": "needs_assessment"
})

graph_builder.add_edge("icebreaker", END)
graph_builder.add_edge("definition", END)
graph_builder.add_edge("quiz", END)
graph_builder.add_edge("examples", END)
graph_builder.add_edge("needs_assessment", END)

graph = graph_builder.compile()

print(" Welcome to the AI Tutoring System!\nLet's get started by getting to know you.\n")

student_name = input("Enter your name: ")
student_std = input("Enter your standard/class: ")
student_topic = input("What topic would you like to learn today? ")

print(f"\nAwesome {student_name}, we're all set to explore {student_topic}!\n")
print("Type 'hi', ask a question, or say 'quiz me' anytime. Type 'quit' to exit.\n")

while True:
    user_input = input("Student (you): ")
    if user_input.lower() in ["quit", "q"]:
        print("👋 Goodbye! Keep learning!")
        break

    state = {
        "messages": [HumanMessage(content=user_input)],
        "name": student_name,
        "standard": student_std,
        "topic": student_topic
    }

    final_state = graph.invoke(state)
    message = final_state["messages"][-1]
    print(f"\nTeacher: {message.content}\n")


 Welcome to the AI Tutoring System!
Let's get started by getting to know you.

Enter your name: John
Enter your standard/class: 6
What topic would you like to learn today? light

Awesome John, we're all set to explore light!

Type 'hi', ask a question, or say 'quiz me' anytime. Type 'quit' to exit.

Student (you): hi

Teacher: Hello John, it's fantastic to meet you. I'm super excited to start this learning journey with you, and today we're going to dive into the amazing world of light. I just know you're going to love discovering all the cool things about light - from rainbows to lasers, and so much more. You're probably already surrounded by light every day, but by the end of our lesson, you'll see it in a whole new way. So, are you ready to shine a light on this fascinating topic and see where it takes us? What's the first thing that comes to mind when you think of light?

Student (you): define light

Teacher: Class.  Today, we're going to talk about light. 

Light is a form of energ